# Constitution d'une base adresse à partir de fichiers DVF
- Adresse complète
- Code commune INSEE
- Code IRIS (code quartier de l'INSEE)
- Coordonnées GPS de la commune
- Coorodnnées GPS de l'adresse

## Lecture du fichier DVF

In [1]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
import time

In [3]:
# Renseigner ici la première ligne et la dernière ligne de la base adresses qui vont être utilisées
row_first = 819100
row_last = 900000

In [4]:
# Chargement du jeu de données
df = pd.read_csv('valeursfoncieres-2020.txt',sep="|",decimal=",")
df.head()

C:\Users\Charles\anaconda3\envs\CDV-perso\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,18,23,24,26,28,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1061.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,BT,NaN,85.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1115.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1940.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1148.0


In [5]:
print(df.columns)

Index(['Code service CH', 'Reference document', '1 Articles CGI',
       '2 Articles CGI', '3 Articles CGI', '4 Articles CGI', '5 Articles CGI',
       'No disposition', 'Date mutation', 'Nature mutation', 'Valeur fonciere',
       'No voie', 'B/T/Q', 'Type de voie', 'Code voie', 'Voie', 'Code postal',
       'Commune', 'Code departement', 'Code commune', 'Prefixe de section',
       'Section', 'No plan', 'No Volume', '1er lot',
       'Surface Carrez du 1er lot', '2eme lot', 'Surface Carrez du 2eme lot',
       '3eme lot', 'Surface Carrez du 3eme lot', '4eme lot',
       'Surface Carrez du 4eme lot', '5eme lot', 'Surface Carrez du 5eme lot',
       'Nombre de lots', 'Code type local', 'Type local', 'Identifiant local',
       'Surface reelle bati', 'Nombre pieces principales', 'Nature culture',
       'Nature culture speciale', 'Surface terrain'],
      dtype='object')


## Constitution d'une base adresse à partir du fichier DVF

In [6]:
# focus on the address datas
ad = df[['No voie', 'Type de voie', 'Voie', 'Code postal','Commune', 'Code departement', 'Code commune']]
ad.shape

(2459560, 7)

In [7]:
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune
0,NaN,NaN,FORTUNAT,1250.0,CEYZERIAT,1,72
1,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1,203
2,NaN,NaN,BOIS DU CHAMP RION,1290.0,LAIZ,1,203
3,NaN,NaN,EN COROBERT,1290.0,LAIZ,1,203
4,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ,1,203


In [8]:
ad['Commune'] = ad['Commune'].replace(['MARSEILLE 1ER', 'MARSEILLE 2EME', 'MARSEILLE 3EME', 'MARSEILLE 4EME',
                                           'MARSEILLE 5EME', 'MARSEILLE 6EME', 'MARSEILLE 7EME', 'MARSEILLE 8EME',
                                           'MARSEILLE 9EME', 'MARSEILLE 10EME', 'MARSEILLE 11EME', 'MARSEILLE 12EME',
                                           'MARSEILLE 13EME', 'MARSEILLE 14EME', 'MARSEILLE 15EME', 'MARSEILLE 16EME',
                                           'LYON 1ER', 'LYON 2EME', 'LYON 3EME', 'LYON 4EME', 'LYON 5EME',
                                           'LYON 6EME', 'LYON 7EME', 'LYON 8EME', 'LYON 9EME',
                                           'PARIS 01', 'PARIS 02', 'PARIS 03', 'PARIS 04', 'PARIS 05', 'PARIS 06',
                                           'PARIS 07', 'PARIS 08', 'PARIS 09', 'PARIS 10', 'PARIS 11', 'PARIS 12',
                                           'PARIS 13', 'PARIS 14', 'PARIS 15', 'PARIS 16', 'PARIS 17', 'PARIS 18',
                                           'PARIS 19', 'PARIS 20'],
                                          ['MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'MARSEILLE', 'MARSEILLE', 'MARSEILLE', 'MARSEILLE',
                                          'LYON', 'LYON', 'LYON', 'LYON', 'LYON',
                                          'LYON', 'LYON', 'LYON', 'LYON',
                                          'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS',
                                          'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS',
                                          'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS', 'PARIS',
                                          'PARIS', 'PARIS'])

In [9]:
ad['Voie'] = ad['Voie'].replace([' L ', ' D '], [' L\'', ' D\''], regex = True)

In [10]:
# On supprime les adresses dupliquées pour éviter d'interroger les API plusieurs fois avec exactement la même adresse
ad = ad.drop_duplicates()
ad.shape

(1059387, 7)

In [11]:
# Traitement des codes postaux (transformés en INT sauf les NA)

ad['Code postal'] = ad['Code postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,1,72
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,1,203
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,1,203
3,NaN,NaN,EN COROBERT,1290,LAIZ,1,203
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,1,289


In [12]:
ad['Code departement'] = ad['Code departement'].apply(lambda x: str(x).zfill(2) if len(str(x)) == 1 else x)
ad['Code commune'] = ad['Code commune'].apply(lambda x: str(x).zfill(3) if len(str(x)) < 3 else x)
ad['Code commune INSEE'] = ad['Code departement'].astype(str) + ad['Code commune'].astype(str)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289


In [13]:
# On formate les adresses
# tools functions : nan values management
def escapeNan(row):
    val = str(row)
    if val == "nan":
        val = " "
    else:
        val = val.replace('.0','') # if from float values
        val = val + " "
    return val

In [14]:
# Création d'un champ "Ville" pour interroger les coordonnées GPS lorsque le champ Adresse ne donne rien
ad['Ville'] = ad['Code postal'] + " " + ad['Commune']
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS


In [15]:
# Création du champ "addressName" qui sera utilisée dans la requête API
ad['Adresse'] = ad.apply(lambda row: 
    escapeNan(row['No voie'])
    + escapeNan(row['Type de voie'])         
    + escapeNan(row['Voie'])
    + escapeNan(row['Code postal']).zfill(5) # pour respecter le format des codes postaux
    + str(row['Commune'])
    , axis = 1).str.strip()

In [16]:
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ,EN COROBERT 1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS


In [17]:
ad['Ville'] = ad['Ville'].replace('-', ' ', regex=True).astype(str)
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse
0,NaN,NaN,FORTUNAT,1250,CEYZERIAT,01,072,01072,1250 CEYZERIAT,FORTUNAT 1250 CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290,LAIZ,01,203,01203,1290 LAIZ,TERRES DES CINQ SAULES 1290 LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290,LAIZ,01,203,01203,1290 LAIZ,BOIS DU CHAMP RION 1290 LAIZ
3,NaN,NaN,EN COROBERT,1290,LAIZ,01,203,01203,1290 LAIZ,EN COROBERT 1290 LAIZ
6,NaN,RUE,DE LA CHARTREUSE,1960,PERONNAS,01,289,01289,1960 PERONNAS,RUE DE LA CHARTREUSE 1960 PERONNAS


## Sélection des adresses à parser pour ajout des coordonnées GPS et codes IRIS

In [18]:
# Sélection des adresses à partir de row_first et row_last définies plus haut
selec = ad.iloc[row_first:row_last, :]
selec.shape

(80900, 10)

In [19]:
# Instanciation de la BDD de correspondance entre Code Postal / Communes et Code INSEE + GPS Communes (nécessaire pour code IRIS)
# La BDD se trouve ici : https://data.opendatasoft.com/explore/dataset/code-postal-code-insee-2015@public/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B
communes = pd.read_csv('databases/code-postal-code-insee-2015.csv', sep = ";")
communes.head()

,Geo Point,Geo Shape,CODE_COM,INSEE_COM,NOM_COM,STATUT,X_CENTROID,Y_CENTROID,Z_MOYEN,SUPERFICIE,POPULATION,CODE_CANT,CODE_ARR,CODE_DEPT,NOM_DEPT,Code_postal,Nom_commune,coordonnees_gps,Unnamed: 18
0,"49.0097923668,0.696073334129","{""type"": ""Polygon"", ""coordinates"": [[[0.682900...",49,27049,BEAUMESNIL,Commune simple,531927,6881409,162,1264,562,2.0,2.0,27,EURE,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297",NaN
1,"49.0521199676,0.412267826158","{""type"": ""Polygon"", ""coordinates"": [[[0.409543...",547,27547,SAINT-GERMAIN-LA-CAMPAGNE,Commune simple,510734,6886673,176,2231,895,3.0,2.0,27,EURE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157",NaN
2,"48.7896521776,0.276557382012","{""type"": ""Polygon"", ""coordinates"": [[[0.260047...",347,61347,RESENLIEU,Commune simple,500194,6858135,231,504,210,21.0,2.0,61,ORNE,61230.0,RESENLIEU,"48.7900887821,0.276930426076",NaN
3,"48.8470682746,0.643176111978","{""type"": ""Polygon"", ""coordinates"": [[[0.628763...",359,27359,JUIGNETTES,Commune simple,527172,6863726,228,1302,243,6.0,3.0,27,EURE,27250.0,JUIGNETTES,"48.8472706788,0.64345379282",NaN
4,"48.8662500091,0.559682765078","{""type"": ""Polygon"", ""coordinates"": [[[0.553791...",136,61136,COUVAINS,Commune simple,521103,6865598,236,1788,167,18.0,2.0,61,ORNE,NaN,NaN,NaN,NaN


In [20]:
# On ne garde que les colonnes qui nous intéressent 
# (On a les coordonnees gps dedans, ce sera intéressant pour faire un check des coordonnees qu'on récupère plus bas)

communes = communes[['INSEE_COM', 'NOM_COM', 'Code_postal', 'Nom_commune', 'coordonnees_gps']]
communes = communes.rename(columns = {'NOM_COM': "Commune"})
communes.head()

,INSEE_COM,Commune,Code_postal,Nom_commune,coordonnees_gps
0,27049,BEAUMESNIL,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297"
1,27547,SAINT-GERMAIN-LA-CAMPAGNE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157"
2,61347,RESENLIEU,61230.0,RESENLIEU,"48.7900887821,0.276930426076"
3,27359,JUIGNETTES,27250.0,JUIGNETTES,"48.8472706788,0.64345379282"
4,61136,COUVAINS,NaN,NaN,NaN


In [21]:
# Instanciation d'un dataframe communes_ville qui ne conserve que le code INSEE, les coordonnés, et la ville
communes_ville = communes
communes_ville['Code_postal'] = communes_ville['Code_postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
communes_ville['Ville'] = communes_ville['Code_postal'] + " " + communes_ville['Commune']
communes_ville['Ville'] = communes_ville['Ville'].where(-communes_ville['Code_postal'].isna(), communes_ville['Commune'])
communes_ville = communes_ville.drop(columns = {'Commune', 'Code_postal', 'Nom_commune'})
communes_ville['Ville'] = communes_ville['Ville'].replace('-', ' ', regex=True).astype(str)
communes_ville.head()

,INSEE_COM,coordonnees_gps,Ville
0,27049,"49.0090520551,0.696415474297",27330 BEAUMESNIL
1,27547,"49.0522921876,0.412852228157",27230 SAINT GERMAIN LA CAMPAGNE
2,61347,"48.7900887821,0.276930426076",61230 RESENLIEU
3,27359,"48.8472706788,0.64345379282",27250 JUIGNETTES
4,61136,NaN,COUVAINS


In [22]:
# Ajout du code commune INSEE et Coordonnées GPS à la sélection d'adresses
selec = selec.merge(communes_ville, on = "Ville", how = "left")
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,coordonnees_gps
0,NaN,NaN,LE VILLAGE,77650,JUTIGNY,77,242,77242,77650 JUTIGNY,LE VILLAGE 77650 JUTIGNY,77242,"48.4940082553,3.22666999554"
1,35.0,AV,VOLTAIRE,77370,NANGIS,77,327,77327,77370 NANGIS,35 AV VOLTAIRE 77370 NANGIS,77327,"48.554637282,3.00883892893"
2,2.0,RUE,GERMAINE TILLION,77370,NANGIS,77,327,77327,77370 NANGIS,2 RUE GERMAINE TILLION 77370 NANGIS,77327,"48.554637282,3.00883892893"
3,14.0,RUE,DE LA REPUBLIQUE,77370,NANGIS,77,327,77327,77370 NANGIS,14 RUE DE LA REPUBLIQUE 77370 NANGIS,77327,"48.554637282,3.00883892893"
4,35.0,AV,DU MARECHAL FOCH,77370,NANGIS,77,327,77327,77370 NANGIS,35 AV DU MARECHAL FOCH 77370 NANGIS,77327,"48.554637282,3.00883892893"


In [23]:
# Test sur Marseille car les codes communes INSEE semblent être différents selon la source
selec[(selec['Ville'].str.contains("MARSEILLE")) & (selec['Code departement'] == 13)]

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,coordonnees_gps


In [24]:
selec['Commune_Lat'] = selec['coordonnees_gps'].str.split(",", expand = True)[0]
selec['Commune_Lon'] = selec['coordonnees_gps'].str.split(",", expand = True)[1]
selec = selec.drop(columns = {'coordonnees_gps'})
selec.head(15)

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon
0,NaN,NaN,LE VILLAGE,77650,JUTIGNY,77,242,77242,77650 JUTIGNY,LE VILLAGE 77650 JUTIGNY,77242,48.4940082553,3.22666999554
1,35.0,AV,VOLTAIRE,77370,NANGIS,77,327,77327,77370 NANGIS,35 AV VOLTAIRE 77370 NANGIS,77327,48.554637282,3.00883892893
2,2.0,RUE,GERMAINE TILLION,77370,NANGIS,77,327,77327,77370 NANGIS,2 RUE GERMAINE TILLION 77370 NANGIS,77327,48.554637282,3.00883892893
3,14.0,RUE,DE LA REPUBLIQUE,77370,NANGIS,77,327,77327,77370 NANGIS,14 RUE DE LA REPUBLIQUE 77370 NANGIS,77327,48.554637282,3.00883892893
4,35.0,AV,DU MARECHAL FOCH,77370,NANGIS,77,327,77327,77370 NANGIS,35 AV DU MARECHAL FOCH 77370 NANGIS,77327,48.554637282,3.00883892893
5,NaN,NaN,LES PRES DU GUE,77370,NANGIS,77,327,77327,77370 NANGIS,LES PRES DU GUE 77370 NANGIS,77327,48.554637282,3.00883892893
6,1.0,RUE,TESSIER,77320,BETON-BAZOCHES,77,032,77032,77320 BETON BAZOCHES,1 RUE TESSIER 77320 BETON-BAZOCHES,77032,48.7072104391,3.24596468954
7,27.0,RUE,GRANDE,77114,GOUAIX,77,208,77208,77114 GOUAIX,27 RUE GRANDE 77114 GOUAIX,77208,48.4760442904,3.2968135463
8,NaN,NaN,LES BARDINES,77160,PROVINS,77,379,77379,77160 PROVINS,LES BARDINES 77160 PROVINS,77379,48.5633093312,3.28756780085
9,13.0,IMP,ANCIENNE LAITERIE,77480,MOUY-SUR-SEINE,77,325,77325,77480 MOUY SUR SEINE,13 IMP ANCIENNE LAITERIE 77480 MOUY-SUR-SEINE,77325,48.4321477503,3.24604389645


In [25]:
# Ne conserver dans code commune INSEE final que Code commune INSEE sauf si > 5 caractères, dans ce cas prendre INSEE_COM
#(DOMTOM ne fonctionne pas sinon pour code iris)

# Problème avec les adresses ayant des apostrophes : non reconnues par geolocator (exemple rue de l'Arcade ...)

In [26]:
selec['code_iris'] = ""
selec['lon'] = ""
selec['lat'] = ""
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Code departement,Code commune,Code commune INSEE,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,code_iris,lon,lat
0,NaN,NaN,LE VILLAGE,77650,JUTIGNY,77,242,77242,77650 JUTIGNY,LE VILLAGE 77650 JUTIGNY,77242,48.4940082553,3.22666999554,,,
1,35.0,AV,VOLTAIRE,77370,NANGIS,77,327,77327,77370 NANGIS,35 AV VOLTAIRE 77370 NANGIS,77327,48.554637282,3.00883892893,,,
2,2.0,RUE,GERMAINE TILLION,77370,NANGIS,77,327,77327,77370 NANGIS,2 RUE GERMAINE TILLION 77370 NANGIS,77327,48.554637282,3.00883892893,,,
3,14.0,RUE,DE LA REPUBLIQUE,77370,NANGIS,77,327,77327,77370 NANGIS,14 RUE DE LA REPUBLIQUE 77370 NANGIS,77327,48.554637282,3.00883892893,,,
4,35.0,AV,DU MARECHAL FOCH,77370,NANGIS,77,327,77327,77370 NANGIS,35 AV DU MARECHAL FOCH 77370 NANGIS,77327,48.554637282,3.00883892893,,,


In [27]:
selec.shape

(80900, 16)

## Récupération des codes IRIS et coordonnées GPS à partir de l'API Nominatim et geo.api.gouv

In [ ]:
# Get GPS Coordinates + IRIS Code from addresses

geolocator = Nominatim(user_agent="pyPredImmoTER")

start = time.time()
etape = time.time()
print("Heure début : ")
print(time.strftime('%H:%M:%S', time.gmtime(start)))
print("\n")
n=0

for row in range(0, row_last - row_first):
    
    iris = ""
    longitude_api = ""
    latitude_api = ""
    longitude_adresse = ""
    latitude_adresse = ""

    
    adresse = selec['Adresse'][row]
    ville = selec['Ville'][row]
    code_INSEE = selec['Code commune INSEE'][row]

    # Interrogation de l'API Nominatim
    location = geolocator.geocode(adresse, country_codes = 'fr', timeout = 5) # On interroge l'API à partir de l'adresse
    
    if location is None: 
        location = geolocator.geocode(ville, country_codes = 'fr', timeout = 5) # Sinon on interroge l'API à partir de la ville
    else:
        latitude_adresse = location.latitude          # On récupère les coordonnées GPS de l'adresse seulement si l'API renvoie qch
        longitude_adresse = location.longitude        
    
    if location is None:                      # Sinon aucun résultat, on ne met rien dans les coordonnées GPS qui vont servir à interroger les codes IRIS
        latitude_api = ""
        longitude_api = ""
    
    else:
        latitude_api = location.latitude
        longitude_api = location.longitude
        
        if isinstance(code_INSEE, float):     # Si pas de code commune
            print(adresse + ': pas de code commune INSEE')
            iris = ""
        elif latitude_api == "":               # Si pas de coordonnées GPS
            print(adresse + ": pas de coordonnees GPS pour code IRIS")
            iris = ""
        else:                              # Sinon on interroge l'API du gouv pour récupérer le code IRIS
            result = requests.get('https://geo.api.gouv.fr/iris?lon=' + str(longitude_api) + '&lat=' 
                                  + str(latitude_api) + '&codeCommune=' + str(code_INSEE), timeout = 5)
            if result.ok == True:
                obj = json.loads(result.text)
                iris = obj['codeIris']
                #print(obj['codeIris'])
            else:
                result.close()
            print(adresse + " ===> " + str(latitude_adresse) + "," + str(longitude_adresse) + " (code IRIS = " + str(iris) + ")")
        
    selec['code_iris'][row] = iris
    selec['lon'][row] = longitude_adresse
    selec['lat'][row] = latitude_adresse
    
    n = n + 1
    
    if (n % 100 == 0):
        print("\n")
        print("Etape : " + str(n))
        print(time.strftime('%H:%M:%S', time.gmtime(time.time())))
        print("Temps pour 100 : " + str(time.time() - etape) + " sec")
        print("\n")
        etape = time.time()

    if (n % 10000 == 0):
        selec.iloc[(n - 10000) : n, :].to_csv(
            'export_gps_iris_' + str(row_first + n - 10000) + "-" + str(row_first + n)+'.csv')
        
end = time.time()
print("\n")
print("Heure fin : ")
print(time.strftime('%H:%M:%S', time.gmtime(end)))
print("\n")

print("Temps de calcul : ")
print(str(end - start) + ' sec')

Heure début : 
15:17:52


LE VILLAGE 77650 JUTIGNY ===> 48.4978612,3.232652 (code IRIS = 772420000)
35 AV VOLTAIRE 77370 NANGIS ===> 48.5560401,3.0190378 (code IRIS = 773270101)
2 RUE GERMAINE TILLION 77370 NANGIS ===> 48.5543564,3.0024955 (code IRIS = 773270103)
14 RUE DE LA REPUBLIQUE 77370 NANGIS ===> 48.5546389,3.016545 (code IRIS = 773270101)
35 AV DU MARECHAL FOCH 77370 NANGIS ===> 48.5609609,3.0133417 (code IRIS = 773270101)
LES PRES DU GUE 77370 NANGIS ===> , (code IRIS = 773270101)
1 RUE TESSIER 77320 BETON-BAZOCHES ===> 48.7006265,3.2434879 (code IRIS = 770320000)
27 RUE GRANDE 77114 GOUAIX ===> 48.4872893,3.2926824 (code IRIS = 772080000)
LES BARDINES 77160 PROVINS ===> 48.5700373,3.3035148 (code IRIS = 773790101)
13 IMP ANCIENNE LAITERIE 77480 MOUY-SUR-SEINE ===> , (code IRIS = 773250000)
4 RUE DES FOSSES 77171 SOURDUN ===> 48.5396547,3.3557812 (code IRIS = 774590000)
5 ALL DES BOUVREUILS 77480 BRAY-SUR-SEINE ===> 48.4119803,3.2435953 (code IRIS = 770510000)
LA SOLINE 77114

17 RUE DU VIEUX PONT 77480 MOUY-SUR-SEINE ===> 48.4200446,3.238593 (code IRIS = 773250000)
L EPINEE 77480 MOUY-SUR-SEINE ===> , (code IRIS = 773250000)
9 RUE DU VIEUX MOULIN 77114 HERME ===> 48.4831881,3.3591355 (code IRIS = 772270000)
10 RUE DES DAMES 77171 SOURDUN ===> 48.5386485,3.3569872 (code IRIS = 774590000)
1 RTE NANGIS 77970 BANNOST-VILLEGAGNON ===> 48.6632935,3.1744317 (code IRIS = 770200000)
RUE DE NANGIS 77970 BANNOST-VILLEGAGNON ===> 48.6632935,3.1744317 (code IRIS = 770200000)
8 RUE AUX CHAMPS 77171 MELZ-SUR-SEINE ===> 48.5088904,3.3912903 (code IRIS = 772890000)
MAULNY 77171 MELZ-SUR-SEINE ===> 48.5084467,3.3916892 (code IRIS = 772890000)
LES FOSSES 77171 MELZ-SUR-SEINE ===> , (code IRIS = 772890000)
6 RUE FARABOEUF 77160 PROVINS ===> 48.5567814,3.2955743 (code IRIS = 773790103)
2 RUE DE ST AIGNAN 77171 LECHELLE ===> , (code IRIS = 772460000)
VOULTON 77560 VOULTON ===> 48.616867,3.333054 (code IRIS = 775300000)
62 AV VICTOR HUGO 77370 NANGIS ===> 48.5582158,3.0113302 (co

9 RUE FORT A FAIRE 77560 COURCHAMP ===> , (code IRIS = 771340000)
2 RUE DE LA PLACE DES FETES 77114 VILLIERS-SUR-SEINE ===> 48.4555624,3.3750678 (code IRIS = 775220000)
18 RUE CREUSE 77160 ROUILLY ===> 48.5881478,3.2860148 (code IRIS = 773910000)
8 RUE DU GENERAL LECLERC 77480 BRAY-SUR-SEINE ===> 48.4164106,3.2379724 (code IRIS = 770510000)
10 RUE DES REMPARTS 77480 BRAY-SUR-SEINE ===> 48.4171595,3.2360749 (code IRIS = 770510000)
RTE DE BRAY 77160 PROVINS ===> 48.5562205,3.2877408 (code IRIS = 773790102)
VEAU GUERIN 77118 BALLOY ===> , (code IRIS = 770190000)
12 RUE DU GROS CAILLOU 77520 DONNEMARIE-DONTILLY ===> 48.4783622,3.1380892 (code IRIS = 771590000)
4 RUE DE L'EGLISE 77520 MONS-EN-MONTOIS ===> 48.4886404,3.1457775 (code IRIS = 772980000)
33 RUE DE VILLENEUVE 77154 COUTENCON ===> 48.4725774,3.0001978 (code IRIS = 771400000)
LE CHAMP DE LA CROIX 77970 BANNOST-VILLEGAGNON ===> , (code IRIS = 770200000)
LA GARE 77970 BANNOST-VILLEGAGNON ===> 48.6798318,3.1902549 (code IRIS = 7702000

LES GRANDS CLOS 77480 MOUSSEAUX-LES-BRAY ===> , (code IRIS = 773210000)
35 RUE DE LA TOUR 77171 LECHELLE ===> 48.5774236,3.3893187 (code IRIS = 772460000)
LE BOURG 77171 LECHELLE ===> 49.190591,1.2344055 (code IRIS = )
RUE DU DOCTEUR SCHWEITZER 77650 SAINTE-COLOMBE ===> 48.5298128,3.2608112 (code IRIS = 774040000)
34 RUE DU DOCTEUR SCHWEITZER 77650 SAINTE-COLOMBE ===> 48.5298128,3.2608112 (code IRIS = 774040000)
MOUSSEAUX LES BRAY 77480 MOUSSEAUX-LES-BRAY ===> 48.4170599,3.230666927122895 (code IRIS = 773210000)
151 CITE DE LA SUCRERIE 77480 MOUSSEAUX-LES-BRAY ===> 48.4147395,3.2303215 (code IRIS = 773210000)
5 CHE DES ORMES 77520 VIMPELLES ===> 48.4597325,3.1647684 (code IRIS = 775240000)
15 RUE DE LA PORTE DE PROVINS 77520 DONNEMARIE-DONTILLY ===> 48.4786514,3.1353627 (code IRIS = 771590000)
25 ALL LIONEL TERRAY 77370 NANGIS ===> 48.5537178,3.0214768 (code IRIS = 773270102)
4 RUE DE LA REPUBLIQUE 77370 NANGIS ===> 48.5546389,3.016545 (code IRIS = 773270101)
LA MORT PLANTY 77118 BAZOC

15 RTE NATIONALE 77134 ORMES-SUR-VOULZIE (LES) ===> 48.4645476,3.2266749 (code IRIS = 773470000)
6 RUE PERREE 77370 MAISON-ROUGE ===> 48.5575252,3.1513352 (code IRIS = 772720000)
7 RUE GRANDE 77560 AUGERS-EN-BRIE ===> 48.68035,3.3565038 (code IRIS = 770120000)
5248  LA GRANDE BOSSE 77118 BAZOCHES LES BRAY ===> , (code IRIS = 770250000)
6 RUE PIERRE EUGENE CLAIRIN 77160 PROVINS ===> 48.5546635,3.2904222 (code IRIS = 773790201)
PRAIRIE ENTRE DEUX EAUX 77160 PROVINS ===> , (code IRIS = 773790103)
420 RUE DE VILLENEUVE 77154 COUTENCON ===> 48.4725774,3.0001978 (code IRIS = 771400000)
13 RTE DE PROVINS 77560 COURTACON ===> 48.6913242,3.290132 (code IRIS = 771370000)
3 RUE DE LA FONTAINE 77520 MEIGNEUX ===> 48.51185815,3.103558177040677 (code IRIS = 772860000)
11 PL DE L'EGLISE 77370 FONTAINS ===> 48.52638,3.0012484 (code IRIS = 771900000)
3 RUE ESTIENNE D'ORVES 77370 NANGIS ===> , (code IRIS = 773270101)
17 RUE DES PRES HAUTS 77370 MAISON-ROUGE ===> 48.5552308,3.1500165 (code IRIS = 7727200

...

77 RUE DU DOCTEUR BASSET 79100 SAINTE-VERGE ===> 47.004799,-0.2158188 (code IRIS = 793000000)
24 RUE DU MOULIN A VENT 79100 THOUARS ===> 46.9469396,-0.1477158 (code IRIS = 793290107)
LA GARLIERE 79200 LA CHAPELLE BERTRAND ===> 46.6212359,-0.189814 (code IRIS = 790710000)
LA TOUR NEUVE 79440 COURLAY ===> , (code IRIS = 791030000)
2  LA TOUR NEUVE 79440 COURLAY ===> , (code IRIS = 791030000)
LE THEUIL 79420 VAUSSEROUX ===> , (code IRIS = 793400000)
5047  LE THEUIL 79420 VAUSSEROUX ===> , (code IRIS = 793400000)
12  L AVENEAU 79340 VASLES ===> , (code IRIS = 793390000)
1 RUE DU VIEUX PUITS 79370 FRESSINES ===> 46.3223816,-0.2966778 (code IRIS = 791290000)
LA CROISIERE 79800 SOUVIGNE ===> , (code IRIS = 793190000)
LA VIEILLE VIGNE 79800 SOUVIGNE ===> , (code IRIS = 793190000)


Etape : 27800
20:08:51
Temps pour 100 : 69.42418599128723 sec


9 RUE DU HUIT MAI 1945 79000 NIORT ===> 46.1778371,-0.4711791 (code IRIS = )
11 RUE DU HUIT MAI 1945 79000 NIORT ===> 46.1778371,-0.4711791 (code IRIS 

CORBAS 79210 ARCAIS ===> , (code IRIS = 790100000)
1 VEN AUX LOUPS 79200 CHATILLON SUR THOUET ===> , (code IRIS = 790800000)
174 RUE DU COMMANDANT CAUD 79360 BEAUVOIR-SUR-NIORT ===> , (code IRIS = 790310000)


Etape : 27900
20:10:01
Temps pour 100 : 69.99265551567078 sec


CHANTELOUP 79000 BESSINES ===> 46.2991418,-0.5299141 (code IRIS = 790340000)
7 IMP LORETTE 79210 MAUZE-SUR-LE-MIGNON ===> 46.1952563,-0.6686084 (code IRIS = 791700000)
141 RUE SAINT-GELAIS 79000 NIORT ===> 46.3253172,-0.460401 (code IRIS = 791910101)
143 RUE SAINT-GELAIS 79000 NIORT ===> 46.3253172,-0.460401 (code IRIS = 791910101)
LA CREUE 79800 SOUVIGNE ===> , (code IRIS = 793190000)
2 CHE DE PAPON 79370 CELLES-SUR-BELLE ===> 46.2552292,-0.212848 (code IRIS = 790610000)
PAPON 79370 CELLES-SUR-BELLE ===> 46.2555433,-0.21287072751201225 (code IRIS = 790610000)
11 RUE DES MERISIERS 79180 CHAURAY ===> 46.3586821,-0.3886611 (code IRIS = 790810102)
60  LE CHEMIN BAS THORIGNE 79370 AIGONDIGNE ===> 46.2918657,-0.2436245 (c

PRES VALLET 79800 SOUVIGNE ===> , (code IRIS = 793190000)
MONT SENIOUX 79800 SOUVIGNE ===> , (code IRIS = 793190000)


Etape : 28000
20:11:03
Temps pour 100 : 62.55310320854187 sec


4 RUE HENRI BATONNIER 79400 ST MAIXENT L ECOLE ===> , (code IRIS = )
LES CHAGNASSES 79800 SOUVIGNE ===> 46.3582402,-0.1581413 (code IRIS = 793190000)
7 CHE DE LA PLAINE DRAHE 79260 LA CRECHE ===> 46.3838405,-0.2873067 (code IRIS = 790480000)
LA POULINIERE 79250 NUEIL-LES-AUBIERS ===> 46.9219658,-0.620992 (code IRIS = 791950000)
44 RTE DE COMBRAND 79250 NUEIL-LES-AUBIERS ===> , (code IRIS = 791950000)
4  BEAUREGARD 79110 LOUBIGNE ===> 46.0868773,-0.0904164 (code IRIS = 791530000)
LES PARCS 79110 LOUBIGNE ===> , (code IRIS = 791530000)
2 IMP DU TRESOR 79120 MESSE ===> 46.2495663,0.1308436 (code IRIS = 791770000)
LAGE 79120 MESSE ===> 46.2598982,0.11082309374798874 (code IRIS = 791770000)
43 RUE DE LA CHAPELLE BONNEUIL 79120 SAINTE-SOLINE ===> 46.2747155,0.0443035 (code IRIS = 792970000)
L OUCHETTE 79120 SAIN

2 RTE DE ST JEAN D'ANGELY CHARZA 79500 MELLE ===> , (code IRIS = 791740000)
5121  PATRE SUD 79600 AVAILLES-THOUARSAIS ===> , (code IRIS = 790220000)
10 IMP GASPARD MONGE 79000 NIORT ===> 46.3096655,-0.4646122 (code IRIS = 791910302)
7  VENELLE MEDIANE 79000 NIORT ===> 46.3282677,-0.4525457814999999 (code IRIS = 791910201)
8  VENELLE DU LEVANT 79000 NIORT ===> 46.3282472,-0.4524622 (code IRIS = 791910201)
27 RUE DE SAINT ANDRE 79380 LA FORET-SUR-SEVRE ===> , (code IRIS = 791230000)
5596  LES TERRIERES 79400 SAIVRES ===> 46.4223962,-0.2132636 (code IRIS = 793020000)
LA COURSAUDIERE OUEST 79340 VASLES ===> , (code IRIS = 793390000)
7000  LA COURSAUDIERE 79340 VASLES ===> 46.6069795,-0.051229 (code IRIS = 793390000)
LES TERRES DE LA BERLIERE 79400 NANTEUIL ===> , (code IRIS = 791890000)
46 IMP BEL AIR 79230 AIFFRES ===> 46.3008972,-0.4240123 (code IRIS = 790030101)
6 RUE PIERRE DE RONSARD 79200 PARTHENAY ===> 46.637650199999996,-0.24954750408163612 (code IRIS = 792020104)
RUE DE LA ROUTIER

BORDEVAIRE 79300 BRESSUIRE ===> 46.833583,-0.4663375 (code IRIS = 790490103)
165 BD DE POITIERS 79300 BRESSUIRE ===> 46.8394268,-0.4882835 (code IRIS = 790490102)
LES CHAMPS DU BOIS ROBINEL 79600 AIRVAULT ===> , (code IRIS = 790050000)
36 RUE DU STADE 79600 BOUSSAIS ===> 46.8081116,-0.2767941 (code IRIS = 790470000)
53 RUE LOUISE MICHEL 79000 NIORT ===> 46.3361809,-0.4381175 (code IRIS = 791910603)
3 CHE DES OUCHETTES 79410 CHERVEUX ===> 46.4122171,-0.3613911 (code IRIS = 790860000)
6 RTE DE L'ARCEAU 79150 SAINT MAURICE ETUSSON ===> 47.0322238,-0.5039379 (code IRIS = 792800000)
140 RTE DES PUITS 79140 BRETIGNOLLES ===> 46.8657497,-0.5925782 (code IRIS = 790500000)
D HOLLEAU 79150 VOULMENTIN ===> , (code IRIS = 792420000)
FRANGEUSE 79150 VOULMENTIN ===> 46.938908,-0.5049166 (code IRIS = 792420000)
PONT GROLLEAU 79150 VOULMENTIN ===> , (code IRIS = 792420000)
5058  FRANGEUSE 79150 VOULMENTIN ===> 46.938908,-0.5049166 (code IRIS = 792420000)
12 RUE DU TAILLIS 79140 CERIZAY ===> , (code IR

In [ ]:
selec.head()

In [ ]:
selec.shape

## Export de la totalité de la sélection d'adresses après la fin du job

In [ ]:
selec.to_csv('export_gps_iris_' + str(row_first) + "-" + str(row_last)+'.csv')